In [1]:
import numpy as np
import random


In [83]:

def sigmoid(Z):
    output = 1 / (1 + np.exp(-Z))
    return output

#This function creates the length and coordinates of the bugs
def initialize_intellect_coordinates(length_of_intellect,random_seed,data_length,num_data):#length of the food. 
    random.seed(random_seed)
    
    intellect = {}
    intellect["coordinates"] = np.ones((4,1))
    intellect["W1"] = np.random.randn(length_of_intellect,data_length)
    intellect["b1"] = np.zeros((length_of_intellect,1))
    intellect["W2"] = np.random.randn(4,length_of_intellect)
    intellect["b2"] = np.zeros((4,1))
    intellect["decider_W3"] = np.random.randn(1,num_data)
    intellect["decider_b3"] = np.zeros((1,1))
    intellect["score"] = 0
    intellect["distance_list"] = []
    return intellect 

#This function creates as many bugs as we want.
def make_bugs(random_seed,num_bugs,length_of_intellect,data_length,num_data):
    random.seed(3)
    bugs = {}
    for i in range(0,num_bugs):
        bugs["bug" + str(i)] = initialize_intellect_coordinates(length_of_intellect,i,data_length,num_data)

    return bugs

#This function give us the inherent features of food and the coordinates of the food.
def initialize_food(random_seed):
    random.seed(random_seed)
    food_features = {}
    food_features["coordinates"] = np.random.randn(4,1) 
    food_features["chemical_volatility"] = np.random.randn(4,1)
    #food_features["merged"] = np.append(food_features["coordinates"] ,food_features["chemical_volatility"])
    return food_features["coordinates"]#food_features["merged"]

#This fuınction create foods as much as we want
def make_food(num_food):
    random.seed(4)
    food = {}
    for f in range(0,num_food):
        food["fo" + str(f)] = initialize_food(f)
    return food

#This function describes the act of a bug thinking of which route to take.
def bug_deciding(bug_name,bugs,food):
    bugy = bugs[str(bug_name)]["coordinates"]
    W1 = bugs[str(bug_name)]["W1"]
    b1 = bugs[str(bug_name)]["b1"]
    W2 = bugs[str(bug_name)]["W2"]
    b2 = bugs[str(bug_name)]["b2"]
    decider_W3 = bugs[str(bug_name)]["decider_W3"]
    decider_b3 = bugs[str(bug_name)]["decider_b3"]
    
    a = food["fo" + str(0)]
    a = a.reshape(a.shape[0],1)
    #print("a is :",a)
    for i in range(1,len(food)):
        b = food["fo"+ str(i)]
        b = b.reshape(b.shape[0],1)
        #print("b is:" , b)
        a = np.append(a,b,axis = 1)
    
    k = np.ones([a.shape[0],a.shape[1]])
    #print("k is: " , k)
    second = k * bugy
    #print("second is : ",second)
    a = np.append(a,second,axis = 0)    
    
    #print("result : ",a)
   
    #this part is the forward propagation part
    
    Z1 = np.dot(W1,a) + b1
    A1 = Z1
    Z2 = np.dot(W2,A1) + b2
    A2 = Z2
    Z3 = np.dot(decider_W3,A2.T) + decider_b3
    AL = Z3
    #so as a result bug looks at all of the data. And returns an array the size(4,#data)
    #So we need to also make it another decision like which way you will pick.
    #we will make a single layer neural network
    return AL.T
    
#This function generalizes deciding to all the bugs
def all_the_bugs_deciding(bugs,food):
    movement_points = {}
    for k in range(len(bugs)):
        movement_points["bug" + str(k)] = bug_deciding("bug" + str(k) , bugs,food)
        
    return movement_points
        

    
#This function let s bugs do their thought moves.    
def move_all_the_bugs(bugs,movement_points):
    for i in range(len(bugs)):
        bugs["bug" + str(i)]["coordinates"] = bugs["bug" + str(i)]["coordinates"] +  movement_points["bug" + str(i)]
    return bugs
  
def evaluating_bugs(bugs,food):
#take a bug and calculate it s distance from all the food. 
    
    for i in range(len(bugs)):
        distance_list = bugs["bug" + str(i)]["distance_list"]
        for j in range(len(food)):
            vector_dif = bugs["bug" + str(i)]["coordinates"] - food["fo" + str(j)]
            distance = (np.dot(vector_dif.T,vector_dif))** 0.5
            distance_list.append(distance)
        bugs["bug" + str(i)]["score"] = min(distance_list)
        bugs["bug" + str(i)]["distance_list"] = []
    
            
        
        
def natural_selection(bugs,food,surv_num = 4):#The top two bugs that get closest to the food gives birth to next generation.
#This function will calculate distance of the bugs to the closest food resources and the closest ones are rewarded with reproduction rights.
#get the coordinate of bug 1 , find distances for all the food,
    evaluating_bugs(bugs,food)
    
    bug_score = {}
    
    for i in range(len(bugs)):
    
        bug_score["bug" + str(i)] = bugs["bug" + str(i)]["score"]
        
    score_sorted = sorted(bug_score.items(), key = lambda x:x[1],reverse=False)
    
    survivor_num = len(bugs) // surv_num
    
    survivors = score_sorted[0:survivor_num]
    survivor_names = []
    for s in range(len(survivors)):
        survivor_names.append(survivors[s][0])
        
    
    return survivor_names

def single_cell_shuffle(bugs,survivor_names,gene_bag):
    #take bug0 and capture all of their cells for layer 1. And then put them in a dictionary for layers
    #bugn cells in layer 1 go to layer_1_cells dictionary.
    #bugn cells in layer 2 go to layer_2_cells dictionary.
    
    gene_bag["layer_1_cells_W1"] = []
    gene_bag["layer_1_cells_b1"] = []
    gene_bag["layer_2_cells_W2"] = []
    gene_bag["layer_2_cells_b2"] = []
    gene_bag["layer_3_cells_W3"] = []
    gene_bag["layer_3_cells_b3"] = []
    for i in range(len(survivor_names)):
        for s in range(len(bugs["bug0"]["W1"])):
            gene_bag["layer_1_cells_W1"].append(bugs[survivor_names[i]]["W1"][s,:])
    for i in range(len(survivor_names)):
        for s in range(len(bugs["bug0"]["b1"])):
            gene_bag["layer_1_cells_b1"].append(bugs[survivor_names[i]]["b1"][s,:])
            
    for i in range(len(survivor_names)):
        for s in range(len(bugs["bug0"]["W2"])):
            gene_bag["layer_2_cells_W2"].append(bugs[survivor_names[i]]["W2"][s,:])
    
    for i in range(len(survivor_names)):
        for s in range(len(bugs["bug0"]["b2"])):
            gene_bag["layer_2_cells_b2"].append(bugs[survivor_names[i]]["b2"][s,:])
    
    for i in range(len(survivor_names)):
        for s in range(len(bugs["bug0"]["decider_W3"])):
            gene_bag["layer_3_cells_W3"].append(bugs[survivor_names[i]]["decider_W3"][s,:])
    
    for i in range(len(survivor_names)):
        for s in range(len(bugs["bug0"]["decider_b3"])):
            gene_bag["layer_3_cells_b3"].append(bugs[survivor_names[i]]["decider_b3"][s,:])
    
    
    return gene_bag

def two_cell_shuffle(bugs,survivor_names,gene_bag):
    
    gene_bag["layer_1_2_cells_W1"] = []
    gene_bag["layer_1_2_cells_b1"] = []
    gene_bag["layer_2_2_cells_W2"] = []
    gene_bag["layer_2_2_cells_b2"] = []
    gene_bag["layer_3_2_cells_W3"] = []
    gene_bag["layer_3_2_cells_b3"] = []
    
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["W1"])),2):
            gene_bag["layer_1_2_cells_W1"].append(bugs[survivor_names[i]]["W1"][s:s+2,:])
            
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["b1"])),2):
            gene_bag["layer_1_2_cells_b1"].append(bugs[survivor_names[i]]["b1"][s:s+2,:])
            
            
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["W2"])),2):
            gene_bag["layer_2_2_cells_W2"].append(bugs[survivor_names[i]]["W2"][s:s+2,:])
            
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["b2"])),2):
            gene_bag["layer_2_2_cells_b2"].append(bugs[survivor_names[i]]["b2"][s:s+2,:])
            
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["decider_W3"])),2):
            gene_bag["layer_3_2_cells_W3"].append(bugs[survivor_names[i]]["decider_W3"][s:s+2,:])
    
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["decider_b3"])),2):
            gene_bag["layer_3_2_cells_b3"].append(bugs[survivor_names[i]]["decider_b3"][s:s+2,:])
            
            
    return gene_bag
        
    
def four_cell_shuffle(bugs,survivor_names,gene_bag):
    gene_bag["layer_1_4_cells_W1"] = []
    gene_bag["layer_1_4_cells_b1"] = []
    gene_bag["layer_2_4_cells_W2"] = []
    gene_bag["layer_2_4_cells_b2"] = []
    gene_bag["layer_3_4_cells_W3"] = []
    gene_bag["layer_3_4_cells_b3"] = []
    
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["W1"])),4):
            gene_bag["layer_1_4_cells_W1"].append(bugs[survivor_names[i]]["W1"][s:s+4,:])
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["b1"])),4):
            gene_bag["layer_1_4_cells_b1"].append(bugs[survivor_names[i]]["b1"][s:s+4,:])
            
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["W2"])),4):
            gene_bag["layer_2_4_cells_W2"].append(bugs[survivor_names[i]]["W2"][s:s+4,:])
            
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["b2"])),4):
            gene_bag["layer_2_4_cells_b2"].append(bugs[survivor_names[i]]["b2"][s:s+4,:])
            
            
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["decider_W3"])),4):
            gene_bag["layer_3_4_cells_W3"].append(bugs[survivor_names[i]]["decider_W3"][s:s+4,:])
            
            
    for i in range(len(survivor_names)):
        for s in range(0,int(len(bugs["bug0"]["decider_b3"])),4):
            gene_bag["layer_3_4_cells_b3"].append(bugs[survivor_names[i]]["decider_b3"][s:s+4,:])
            
            
    
    return gene_bag

def the_big_shuffler(bugs,survivor_names):
    gene_bag = {}
    
    gene_bag = single_cell_shuffle(bugs,survivor_names,gene_bag)
    
    gene_bag = two_cell_shuffle(bugs,survivor_names,gene_bag)
    
    gene_bag = four_cell_shuffle(bugs,survivor_names,gene_bag)
    
    return gene_bag


def single_cell_create(bugs,gene_bag):
    for i in range(0,len(bugs)//2):
        gene_W1 = []
        gene_b1 = []
        gene_W2 = []
        gene_b2 = []
        gene_W3 = []
        gene_b3 = []
        for k in range(len(bugs["bug0"]["W1"])):
            a = random.randint(0,len(gene_bag["layer_1_cells_W1"])-1)
            gene_W1.append(gene_bag["layer_1_cells_W1"][a])
             
            b = random.randint(0,len(gene_bag["layer_1_cells_b1"])-1)
            gene_b1.append(gene_bag["layer_1_cells_b1"][b])                
            
        for s in range(len(bugs["bug0"]["W2"])):
            c = random.randint(0,len(gene_bag["layer_2_cells_W2"])-1)
            gene_W2.append(gene_bag["layer_2_cells_W2"][c])
            
            d = random.randint(0,len(gene_bag["layer_2_cells_b2"])-1)
            gene_b2.append(gene_bag["layer_2_cells_b2"][d])
            
        for f in range(len(bugs["bug0"]["decider_W3"])):
            e = random.randint(0,len(gene_bag["layer_3_cells_W3"])-1)
            gene_W3.append(gene_bag["layer_3_cells_W3"][e])
            
            f = random.randint(0,len(gene_bag["layer_3_cells_b3"])-1)
            gene_b3.append(gene_bag["layer_3_cells_b3"][f])
                
        
        bugs["bug" + str(i)]["W1"] = np.vstack(gene_W1)
        bugs["bug" + str(i)]["b1"] = np.vstack(gene_b1)
        bugs["bug" + str(i)]["W2"] = np.vstack(gene_W2)
        bugs["bug" + str(i)]["b2"] = np.vstack(gene_b2)
        bugs["bug" + str(i)]["decider_W3"] = np.vstack(gene_W3)
        bugs["bug" + str(i)]["decider_b3"] = np.vstack(gene_b3)
        bugs["bug" + str(i)]["coordinates"] = np.ones((4,1))    
    return bugs



def  two_cell_create(bugs,gene_bag):
    for i in range(len(bugs) // 2,len(bugs) // 2 + (len(bugs) // 2 ) // 2):
        gene_W1 = []
        gene_b1 = []
        gene_W2 = []
        gene_b2 = []
        gene_W3 = []
        gene_b3 = []
        
        for k in range(len(bugs["bug0"]["W1"]) // 2):
            a = random.randint(0,len(gene_bag["layer_1_2_cells_W1"])-1)
            gene_W1.append(gene_bag["layer_1_2_cells_W1"][a])
                
            b = random.randint(0,len(gene_bag["layer_1_2_cells_b1"])-1)
            gene_b1.append(gene_bag["layer_1_2_cells_b1"][b])
            
        for s in range(len(bugs["bug0"]["W2"]) // 2):
        
            c = random.randint(0,len(gene_bag["layer_2_2_cells_W2"])-1)
            gene_W2.append(gene_bag["layer_2_2_cells_W2"][c])
            
            d = random.randint(0,len(gene_bag["layer_2_2_cells_b2"])-1)
            gene_b2.append(gene_bag["layer_2_2_cells_b2"][d])
            
        for j in range(0,len(bugs["bug0"]["decider_W3"]) // 2):
            e = random.randint(0,len(gene_bag["layer_3_2_cells_W3"])-1)
            gene_W3.append(gene_bag["layer_3_2_cells_W3"][e])
            
            f = random.randint(0,len(gene_bag["layer_3_2_cells_b3"])-1)
            gene_b3.append(gene_bag["layer_3_2_cells_b3"][f])
    
    
    
        bugs["bug" + str(i)]["W1"] = np.vstack(gene_W1)
        bugs["bug" + str(i)]["b1"] = np.vstack(gene_b1)
        bugs["bug" + str(i)]["W2"] = np.vstack(gene_W2)
        bugs["bug" + str(i)]["b2"] = np.vstack(gene_b2)
        #bugs["bug" + str(i)]["decider_W3"] = np.vstack(gene_W3)
        #bugs["bug" + str(i)]["decider_b3"] = np.vstack(gene_b3)
        bugs["bug" + str(i)]["coordinates"] = np.ones((4,1))
        
    return bugs
    
    
    
def four_cell_create(bugs,gene_bag):
        for i in range(len(bugs) // 2 + (len(bugs) // 2 ) // 2 , len(bugs) // 2 + (len(bugs) // 2 ) // 2 + (len(bugs) // 2) // 2):
            gene_W1 = []
            gene_b1 = []
            gene_W2 = []
            gene_b2 = []
            gene_W3 = []
            gene_b3 = []
            
            
            for k in range(len(bugs["bug0"]["W1"]) // 4):
                a = random.randint(0,len(gene_bag["layer_1_4_cells_W1"])-1)
                gene_W1.append(gene_bag["layer_1_4_cells_W1"][a])
                
                b = random.randint(0,len(gene_bag["layer_1_4_cells_b1"])-1)
                gene_b1.append(gene_bag["layer_1_4_cells_b1"][b])
            
            for s in range(len(bugs["bug0"]["W2"]) // 4):
        
                c = random.randint(0,len(gene_bag["layer_2_4_cells_W2"])-1)
                gene_W2.append(gene_bag["layer_2_4_cells_W2"][c])
            
                d = random.randint(0,len(gene_bag["layer_2_4_cells_b2"])-1)
                gene_b2.append(gene_bag["layer_2_4_cells_b2"][d])
            
            for j in range(0,len(bugs["bug0"]["decider_W3"]) // 4):
                e = random.randint(0,len(gene_bag["layer_3_4_cells_W3"])-1)
                gene_W3.append(gene_bag["layer_3_4_cells_W3"][e])
            
                f = random.randint(0,len(gene_bag["layer_3_4_cells_b3"])-1)
                gene_b3.append(gene_bag["layer_3_4_cells_b3"][f])
            
            
            
            bugs["bug" + str(i)]["W1"] = np.vstack(gene_W1)
            bugs["bug" + str(i)]["b1"] = np.vstack(gene_b1)
            bugs["bug" + str(i)]["W2"] = np.vstack(gene_W2)
            bugs["bug" + str(i)]["b2"] = np.vstack(gene_b2)
            #bugs["bug" + str(i)]["decider_W3"] = np.vstack(gene_W3)
            #bugs["bug" + str(i)]["decider_b3"] = np.vstack(gene_b3)
            bugs["bug" + str(i)]["coordinates"] = np.ones((4,1))
            
        return bugs
        
def start_show(bugs,food,iteration_number):
    for i in range(0,iteration_number + 1):
        
        movement_points = all_the_bugs_deciding(bugs,food)
        #print(movement_points)


        #print("beginning: " ,bugs["bug0"]["coordinates"])
        move_all_the_bugs(bugs,movement_points)
        #print("last: " ,bugs["bug0"]["coordinates"])
        #print("movement:" ,movement_points["bug0"])

        evaluating_bugs(bugs,food)

        survivor_names = natural_selection(bugs,food)
        print(survivor_names)
       
        if i < iteration_number:
            gene_bag = the_big_shuffler(bugs,survivor_names)
        #print(gene_bag)

            bugs = single_cell_create(bugs,gene_bag)

            bugs = two_cell_create(bugs,gene_bag)

            bugs = four_cell_create(bugs,gene_bag)

    return bugs,survivor_names
    
    
#This function will assemble bugs from scracth. It will randomly choose bugs intellect parts and unite them in a unique intellect.



    

In [94]:
food = make_food(10)
data_length =food["fo0"].shape[0] + 4 #(This 4 is symbolizes the four direction of movement)
num_data = len(food)

bugs = make_bugs(1,2400,8,data_length,num_data)
#print(bugs["bug0"])
bugs,survivor_names = start_show(bugs,food,100)
#print(bugs["bug0"])
print(survivor_names)

['bug824', 'bug1405', 'bug1572', 'bug1016', 'bug1689', 'bug1864', 'bug1757', 'bug1077', 'bug2140', 'bug279', 'bug183', 'bug1192', 'bug338', 'bug177', 'bug293', 'bug1111', 'bug1049', 'bug1218', 'bug2180', 'bug556', 'bug1332', 'bug1038', 'bug1090', 'bug1491', 'bug1320', 'bug576', 'bug1294', 'bug2370', 'bug431', 'bug705', 'bug13', 'bug2399', 'bug1173', 'bug834', 'bug304', 'bug2117', 'bug735', 'bug1705', 'bug1797', 'bug1411', 'bug842', 'bug1477', 'bug631', 'bug1215', 'bug317', 'bug502', 'bug1839', 'bug240', 'bug19', 'bug744', 'bug2146', 'bug577', 'bug2238', 'bug1475', 'bug1321', 'bug17', 'bug1549', 'bug998', 'bug1794', 'bug1440', 'bug2343', 'bug2042', 'bug1333', 'bug1790', 'bug372', 'bug825', 'bug1895', 'bug990', 'bug2341', 'bug1316', 'bug1782', 'bug753', 'bug1357', 'bug2190', 'bug574', 'bug1652', 'bug1318', 'bug854', 'bug765', 'bug1074', 'bug1831', 'bug1898', 'bug2366', 'bug353', 'bug2122', 'bug2011', 'bug737', 'bug1950', 'bug2162', 'bug776', 'bug1542', 'bug1656', 'bug1348', 'bug2193', 'b

['bug411', 'bug560', 'bug814', 'bug754', 'bug381', 'bug975', 'bug666', 'bug961', 'bug563', 'bug585', 'bug15', 'bug182', 'bug931', 'bug447', 'bug859', 'bug407', 'bug1434', 'bug270', 'bug367', 'bug1151', 'bug112', 'bug778', 'bug394', 'bug2399', 'bug1044', 'bug115', 'bug723', 'bug594', 'bug35', 'bug1025', 'bug148', 'bug328', 'bug113', 'bug185', 'bug932', 'bug1491', 'bug47', 'bug140', 'bug1088', 'bug326', 'bug830', 'bug1156', 'bug1244', 'bug1475', 'bug502', 'bug688', 'bug1572', 'bug575', 'bug402', 'bug810', 'bug350', 'bug581', 'bug1291', 'bug276', 'bug278', 'bug903', 'bug559', 'bug311', 'bug515', 'bug1019', 'bug1705', 'bug747', 'bug1789', 'bug482', 'bug541', 'bug537', 'bug681', 'bug55', 'bug840', 'bug427', 'bug1745', 'bug970', 'bug758', 'bug1045', 'bug497', 'bug302', 'bug1910', 'bug946', 'bug1797', 'bug39', 'bug925', 'bug608', 'bug387', 'bug912', 'bug1179', 'bug877', 'bug1508', 'bug213', 'bug2005', 'bug342', 'bug915', 'bug1149', 'bug766', 'bug555', 'bug294', 'bug146', 'bug1105', 'bug712', 

['bug913', 'bug923', 'bug1146', 'bug129', 'bug456', 'bug126', 'bug540', 'bug781', 'bug84', 'bug163', 'bug514', 'bug583', 'bug1135', 'bug175', 'bug1162', 'bug1088', 'bug857', 'bug231', 'bug2095', 'bug727', 'bug746', 'bug1155', 'bug809', 'bug82', 'bug998', 'bug589', 'bug1110', 'bug411', 'bug1137', 'bug370', 'bug1048', 'bug364', 'bug649', 'bug560', 'bug686', 'bug117', 'bug1149', 'bug827', 'bug674', 'bug54', 'bug1042', 'bug643', 'bug2093', 'bug158', 'bug132', 'bug1703', 'bug1014', 'bug791', 'bug1887', 'bug572', 'bug1034', 'bug1424', 'bug631', 'bug554', 'bug684', 'bug697', 'bug559', 'bug1236', 'bug2199', 'bug315', 'bug807', 'bug208', 'bug136', 'bug896', 'bug938', 'bug346', 'bug1027', 'bug212', 'bug828', 'bug1884', 'bug15', 'bug219', 'bug581', 'bug892', 'bug480', 'bug1013', 'bug264', 'bug1108', 'bug609', 'bug193', 'bug426', 'bug1120', 'bug420', 'bug8', 'bug207', 'bug823', 'bug106', 'bug1190', 'bug1032', 'bug217', 'bug1171', 'bug1350', 'bug1705', 'bug2317', 'bug1059', 'bug761', 'bug1053', 'bu

['bug518', 'bug1110', 'bug1114', 'bug2115', 'bug1009', 'bug1120', 'bug890', 'bug571', 'bug1294', 'bug288', 'bug722', 'bug1076', 'bug49', 'bug449', 'bug600', 'bug1175', 'bug287', 'bug783', 'bug619', 'bug426', 'bug506', 'bug1049', 'bug95', 'bug533', 'bug705', 'bug792', 'bug1119', 'bug372', 'bug391', 'bug827', 'bug1086', 'bug69', 'bug573', 'bug24', 'bug398', 'bug52', 'bug245', 'bug1132', 'bug931', 'bug393', 'bug158', 'bug1079', 'bug529', 'bug101', 'bug165', 'bug355', 'bug98', 'bug236', 'bug971', 'bug108', 'bug698', 'bug855', 'bug244', 'bug59', 'bug1034', 'bug538', 'bug491', 'bug1069', 'bug233', 'bug631', 'bug1168', 'bug2042', 'bug73', 'bug238', 'bug900', 'bug603', 'bug1043', 'bug444', 'bug790', 'bug1185', 'bug78', 'bug844', 'bug1029', 'bug423', 'bug421', 'bug892', 'bug66', 'bug984', 'bug540', 'bug729', 'bug1041', 'bug265', 'bug588', 'bug831', 'bug493', 'bug173', 'bug96', 'bug913', 'bug476', 'bug274', 'bug591', 'bug13', 'bug697', 'bug645', 'bug810', 'bug202', 'bug351', 'bug44', 'bug189', '

['bug822', 'bug415', 'bug157', 'bug582', 'bug889', 'bug413', 'bug68', 'bug46', 'bug895', 'bug580', 'bug1067', 'bug104', 'bug927', 'bug403', 'bug1155', 'bug1056', 'bug367', 'bug217', 'bug81', 'bug606', 'bug1058', 'bug74', 'bug1834', 'bug350', 'bug395', 'bug679', 'bug943', 'bug811', 'bug410', 'bug214', 'bug719', 'bug697', 'bug980', 'bug32', 'bug923', 'bug526', 'bug637', 'bug1594', 'bug1115', 'bug1077', 'bug746', 'bug862', 'bug294', 'bug806', 'bug1061', 'bug828', 'bug1162', 'bug795', 'bug944', 'bug500', 'bug55', 'bug1035', 'bug1132', 'bug733', 'bug14', 'bug1082', 'bug176', 'bug658', 'bug365', 'bug451', 'bug647', 'bug1074', 'bug940', 'bug507', 'bug932', 'bug460', 'bug465', 'bug525', 'bug1079', 'bug608', 'bug666', 'bug642', 'bug420', 'bug324', 'bug1065', 'bug482', 'bug910', 'bug387', 'bug1110', 'bug1116', 'bug1186', 'bug997', 'bug376', 'bug1101', 'bug218', 'bug314', 'bug2399', 'bug1003', 'bug1103', 'bug934', 'bug502', 'bug586', 'bug374', 'bug958', 'bug682', 'bug655', 'bug725', 'bug708', 'bu

...

['bug1901', 'bug612', 'bug656', 'bug233', 'bug354', 'bug247', 'bug865', 'bug380', 'bug49', 'bug998', 'bug522', 'bug79', 'bug624', 'bug1048', 'bug914', 'bug927', 'bug1019', 'bug997', 'bug1010', 'bug444', 'bug363', 'bug821', 'bug408', 'bug226', 'bug356', 'bug758', 'bug203', 'bug684', 'bug731', 'bug825', 'bug288', 'bug641', 'bug945', 'bug1102', 'bug665', 'bug454', 'bug210', 'bug1137', 'bug1068', 'bug480', 'bug156', 'bug341', 'bug566', 'bug1168', 'bug1000', 'bug854', 'bug21', 'bug870', 'bug916', 'bug507', 'bug955', 'bug18', 'bug179', 'bug295', 'bug1079', 'bug639', 'bug818', 'bug623', 'bug757', 'bug1037', 'bug207', 'bug703', 'bug725', 'bug676', 'bug228', 'bug1786', 'bug342', 'bug73', 'bug15', 'bug11', 'bug931', 'bug208', 'bug358', 'bug302', 'bug221', 'bug140', 'bug537', 'bug883', 'bug1064', 'bug1153', 'bug595', 'bug0', 'bug188', 'bug10', 'bug693', 'bug102', 'bug479', 'bug53', 'bug529', 'bug6', 'bug582', 'bug71', 'bug622', 'bug805', 'bug1138', 'bug970', 'bug364', 'bug1186', 'bug647', 'bug95'

['bug505', 'bug848', 'bug459', 'bug220', 'bug75', 'bug190', 'bug86', 'bug460', 'bug914', 'bug169', 'bug296', 'bug399', 'bug424', 'bug68', 'bug736', 'bug1123', 'bug3', 'bug1059', 'bug272', 'bug301', 'bug207', 'bug1127', 'bug56', 'bug568', 'bug249', 'bug175', 'bug375', 'bug233', 'bug759', 'bug922', 'bug676', 'bug217', 'bug556', 'bug396', 'bug973', 'bug162', 'bug639', 'bug1023', 'bug154', 'bug1094', 'bug582', 'bug939', 'bug293', 'bug52', 'bug1196', 'bug637', 'bug181', 'bug1188', 'bug411', 'bug429', 'bug735', 'bug378', 'bug222', 'bug1192', 'bug686', 'bug804', 'bug982', 'bug230', 'bug1093', 'bug1041', 'bug913', 'bug297', 'bug1144', 'bug134', 'bug472', 'bug246', 'bug849', 'bug1136', 'bug120', 'bug1169', 'bug792', 'bug705', 'bug65', 'bug463', 'bug1379', 'bug935', 'bug490', 'bug651', 'bug70', 'bug1083', 'bug1033', 'bug718', 'bug617', 'bug755', 'bug82', 'bug319', 'bug304', 'bug94', 'bug143', 'bug1070', 'bug413', 'bug265', 'bug599', 'bug881', 'bug561', 'bug1116', 'bug340', 'bug332', 'bug780', 'b

['bug1175', 'bug1777', 'bug115', 'bug412', 'bug251', 'bug861', 'bug230', 'bug219', 'bug2274', 'bug374', 'bug987', 'bug680', 'bug1085', 'bug441', 'bug682', 'bug905', 'bug908', 'bug231', 'bug396', 'bug789', 'bug859', 'bug742', 'bug1048', 'bug811', 'bug247', 'bug75', 'bug1171', 'bug824', 'bug1137', 'bug497', 'bug909', 'bug1153', 'bug438', 'bug378', 'bug449', 'bug1185', 'bug437', 'bug548', 'bug165', 'bug459', 'bug483', 'bug1075', 'bug873', 'bug298', 'bug398', 'bug595', 'bug719', 'bug572', 'bug241', 'bug372', 'bug643', 'bug451', 'bug941', 'bug73', 'bug815', 'bug1008', 'bug108', 'bug982', 'bug464', 'bug534', 'bug812', 'bug1145', 'bug888', 'bug679', 'bug405', 'bug915', 'bug749', 'bug1046', 'bug7', 'bug1052', 'bug951', 'bug945', 'bug684', 'bug609', 'bug43', 'bug160', 'bug125', 'bug1014', 'bug797', 'bug269', 'bug949', 'bug630', 'bug1095', 'bug51', 'bug68', 'bug490', 'bug83', 'bug302', 'bug14', 'bug1063', 'bug831', 'bug503', 'bug432', 'bug198', 'bug191', 'bug618', 'bug750', 'bug471', 'bug61', 'b

['bug154', 'bug1594', 'bug1166', 'bug724', 'bug831', 'bug628', 'bug1458', 'bug544', 'bug24', 'bug54', 'bug563', 'bug668', 'bug575', 'bug508', 'bug1112', 'bug529', 'bug483', 'bug613', 'bug1045', 'bug498', 'bug970', 'bug393', 'bug1007', 'bug644', 'bug515', 'bug282', 'bug158', 'bug440', 'bug681', 'bug889', 'bug592', 'bug473', 'bug907', 'bug525', 'bug579', 'bug1120', 'bug1037', 'bug701', 'bug627', 'bug1142', 'bug251', 'bug940', 'bug1165', 'bug780', 'bug203', 'bug220', 'bug1082', 'bug478', 'bug633', 'bug615', 'bug608', 'bug1198', 'bug250', 'bug369', 'bug865', 'bug828', 'bug987', 'bug882', 'bug28', 'bug1128', 'bug807', 'bug376', 'bug444', 'bug2162', 'bug926', 'bug268', 'bug915', 'bug1172', 'bug407', 'bug914', 'bug382', 'bug1054', 'bug586', 'bug94', 'bug1099', 'bug291', 'bug986', 'bug909', 'bug1143', 'bug1048', 'bug368', 'bug852', 'bug1113', 'bug1059', 'bug427', 'bug217', 'bug27', 'bug212', 'bug332', 'bug620', 'bug540', 'bug811', 'bug55', 'bug316', 'bug635', 'bug470', 'bug246', 'bug583', 'bug

['bug258', 'bug1278', 'bug596', 'bug262', 'bug1077', 'bug1872', 'bug851', 'bug1054', 'bug804', 'bug846', 'bug277', 'bug736', 'bug791', 'bug419', 'bug617', 'bug825', 'bug997', 'bug50', 'bug25', 'bug118', 'bug412', 'bug1020', 'bug1038', 'bug263', 'bug411', 'bug687', 'bug223', 'bug605', 'bug467', 'bug417', 'bug857', 'bug923', 'bug2399', 'bug976', 'bug991', 'bug89', 'bug954', 'bug1089', 'bug283', 'bug1004', 'bug31', 'bug788', 'bug696', 'bug1107', 'bug877', 'bug613', 'bug425', 'bug692', 'bug882', 'bug758', 'bug159', 'bug912', 'bug943', 'bug835', 'bug891', 'bug800', 'bug865', 'bug745', 'bug859', 'bug197', 'bug583', 'bug1006', 'bug965', 'bug1141', 'bug1078', 'bug1056', 'bug363', 'bug1007', 'bug833', 'bug323', 'bug463', 'bug445', 'bug1183', 'bug361', 'bug1066', 'bug92', 'bug429', 'bug641', 'bug675', 'bug983', 'bug10', 'bug953', 'bug132', 'bug1034', 'bug1039', 'bug936', 'bug870', 'bug394', 'bug628', 'bug278', 'bug337', 'bug2092', 'bug834', 'bug863', 'bug948', 'bug330', 'bug1057', 'bug389', 'bug

In [95]:
for i in range(len(bugs)):
    print(bugs["bug"+str(i)]["score"])


[[0.89693264]]
[[1.28959084]]
[[0.83946771]]
[[0.8385097]]
[[0.83336773]]
[[0.93198815]]
[[0.83013617]]
[[0.89545275]]
[[0.82979183]]
[[0.85240443]]
[[0.70635009]]
[[1.24210032]]
[[0.8443256]]
[[0.80868388]]
[[0.83948896]]
[[3.7794995]]
[[0.85436164]]
[[0.83952651]]
[[0.81399442]]
[[0.84997197]]
[[0.84774152]]
[[0.84180075]]
[[1.37297432]]
[[0.82250768]]
[[0.84181826]]
[[0.64602654]]
[[1.3036248]]
[[1.39847197]]
[[1.33966667]]
[[0.97650064]]
[[2.24976634]]
[[0.66645177]]
[[0.84558909]]
[[0.83517404]]
[[0.84356433]]
[[0.73414561]]
[[0.84018975]]
[[0.83187864]]
[[0.76859119]]
[[0.8447493]]
[[0.89966415]]
[[0.90336231]]
[[0.84333857]]
[[0.83860867]]
[[0.89938024]]
[[0.8669003]]
[[0.83852776]]
[[0.85209092]]
[[1.44268361]]
[[0.83875435]]
[[0.64588283]]
[[0.83721394]]
[[0.73449905]]
[[0.87163938]]
[[0.83849841]]
[[1.39451472]]
[[0.85565291]]
[[0.83143287]]
[[0.84523989]]
[[0.85429417]]
[[0.88902706]]
[[0.8488555]]
[[0.84300873]]
[[0.83849579]]
[[0.83256785]]
[[0.74521377]]
[[1.37321182]]
[[

[[0.84357681]]
[[0.84194899]]
[[0.83849919]]
[[0.92153671]]
[[0.78887983]]
[[0.88019596]]
[[0.68497229]]
[[0.84772648]]
[[0.88003117]]
[[0.82199761]]
[[3.96967098]]
[[0.81494707]]
[[0.83816342]]
[[0.91768248]]
[[0.84272607]]
[[0.88357009]]
[[0.8290735]]
[[1.15438855]]
[[0.94615493]]
[[0.84082774]]
[[0.84516635]]
[[0.84019593]]
[[0.83973261]]
[[0.83373986]]
[[0.83935324]]
[[0.84909515]]
[[0.83856045]]
[[0.68119708]]
[[0.83849081]]
[[0.83850178]]
[[0.90028529]]
[[0.84332737]]
[[0.8387511]]
[[0.740656]]
[[0.83876396]]
[[0.80133134]]
[[0.83861229]]
[[0.87020013]]
[[0.65999243]]
[[0.84812026]]
[[0.85240648]]
[[0.84867798]]
[[1.16942635]]
[[0.86005942]]
[[0.84491348]]
[[0.85214449]]
[[1.33025358]]
[[0.84041093]]
[[0.86728573]]
[[0.93140699]]
[[1.34860333]]
[[0.7076896]]
[[0.83226549]]
[[0.84830862]]
[[0.84364148]]
[[0.80825001]]
[[0.84568494]]
[[0.8599516]]
[[0.6837406]]
[[0.86838528]]
[[0.8320624]]
[[1.1649497]]
[[0.83857367]]
[[0.72062272]]
[[0.83409725]]
[[0.76230208]]
[[0.84360607]]
[[0.

[[11.51061884]]
[[10.34387163]]
[[34.11512611]]
[[0.58010678]]
[[2.25221133]]
[[27.41596339]]
[[19.20906394]]
[[0.84568253]]
[[2.16513291]]
[[5.65157195]]
[[7.06702847]]
[[1.42105933]]
[[17.29191469]]
[[21.20184479]]
[[0.87068785]]
[[1.20024297]]
[[0.81372979]]
[[12.13698384]]
[[7.93254442]]
[[4.39656234]]
[[31.03006196]]
[[3.67066886]]
[[34.42885817]]
[[13.14592845]]
[[14.81248339]]
[[24.09866416]]
[[2.1491685]]
[[12.23574447]]
[[20.56187508]]
[[1.55084737]]
[[26.94872374]]
[[13.65350338]]
[[4.81939382]]
[[26.56651689]]
[[20.79858763]]
[[28.97404832]]
[[3.1283224]]
[[10.69179107]]
[[12.18813497]]
[[1.785638]]
[[32.84322363]]
[[10.67215723]]
[[16.80521982]]
[[14.40867502]]
[[5.96482904]]
[[11.0639786]]
[[7.88291123]]
[[11.26930972]]
[[4.58816505]]
[[22.44667292]]
[[43.82977304]]
[[16.74887225]]
[[19.08711379]]
[[22.06477458]]
[[4.96062991]]
[[11.8296767]]
[[39.3678498]]
[[17.48313531]]
[[45.32885902]]
[[19.99825294]]
[[1.90595956]]
[[9.27905794]]
[[8.46076685]]
[[30.23886201]]
[[4.6883

In [22]:

food = make_food(10)
data_length =food["fo0"].shape[0] + 4 #(This 4 is symbolizes the four direction of movement)
num_data = len(food)

bugs = make_bugs(1,48,4,data_length,num_data)


'''print("Gen W1" ,bugs["bug0"]["W1"])
print("Gen b1",bugs["bug0"]["b1"])
print("Gen W2", bugs["bug0"]["W2"])
print("Gen b2", bugs["bug0"]["b2"])
print("Gen W3", bugs["bug0"]["decider_W3"])
print("Gen b3", bugs["bug0"]["decider_b3"])'''


'''print("Gen W1" ,bugs["bug22"]["W1"])
print("Gen b1",bugs["bug22"]["b1"])
print("Gen W2", bugs["bug22"]["W2"])
print("Gen b2", bugs["bug22"]["b2"])
print("Gen W3", bugs["bug22"]["decider_W3"])
print("Gen b3", bugs["bug22"]["decider_b3"])
'''

print("Gen W1" ,bugs["bug41"]["W1"])
print("Gen b1",bugs["bug41"]["b1"])
print("Gen W2", bugs["bug41"]["W2"])
print("Gen b2", bugs["bug41"]["b2"])
print("Gen W3", bugs["bug41"]["decider_W3"])
print("Gen b3", bugs["bug41"]["decider_b3"])


movement_points = all_the_bugs_deciding(bugs,food)
#print(movement_points)


#print("beginning: " ,bugs["bug0"]["coordinates"])
move_all_the_bugs(bugs,movement_points)
#print("last: " ,bugs["bug0"]["coordinates"])
#print("movement:" ,movement_points["bug0"])

evaluating_bugs(bugs,food)

survivor_names = natural_selection(bugs,food)
#print(survivor_names)

gene_bag = the_big_shuffler(bugs,survivor_names)
#print(gene_bag)

bugs = single_cell_create(bugs,gene_bag)
'''print("nextGen W1" ,bugs["bug0"]["W1"])
print("nextGen b1",bugs["bug0"]["b1"])
print("nextGen W2", bugs["bug0"]["W2"])
print("nextGen b2", bugs["bug0"]["b2"])
print("nextGen W3", bugs["bug0"]["decider_W3"])
print("nextGen b3", bugs["bug0"]["decider_b3"])'''


bugs = two_cell_create(bugs,gene_bag)

'''print("nextGen W1" ,bugs["bug22"]["W1"])
print("nextGen b1",bugs["bug22"]["b1"])
print("nextGen W2", bugs["bug22"]["W2"])
print("nextGen b2", bugs["bug22"]["b2"])
print("nextGen W3", bugs["bug22"]["decider_W3"])
print("nextGen b3", bugs["bug22"]["decider_b3"])
'''

bugs = four_cell_create(bugs,gene_bag)

print("nextGen W1" ,bugs["bug41"]["W1"])
print("nextGen b1",bugs["bug41"]["b1"])
print("nextGen W2", bugs["bug41"]["W2"])
print("nextGen b2", bugs["bug41"]["b2"])
print("nextGen W3", bugs["bug41"]["decider_W3"])
print("nextGen b3", bugs["bug41"]["decider_b3"])



Gen W1 [[-0.4004796  -1.26129403 -0.449756   -0.02261191 -0.71332656 -0.76475869
  -0.60024928 -0.66130513]
 [-0.77903903  0.99843736  1.04720988 -0.06004121  1.48422958  0.34050947
  -0.01367009  0.66446829]
 [ 0.72178606 -0.40635409 -0.68991746 -0.22515122 -2.10337327 -1.28069867
   0.16403747 -0.95770685]
 [-1.36467884  0.57184914  1.33461932  0.1010944   0.77626857 -0.86912554
  -0.68868606  2.59231445]]
Gen b1 [[0.]
 [0.]
 [0.]
 [0.]]
Gen W2 [[-0.91386424 -2.18081946 -0.66123703 -0.06840691]
 [ 1.77884592 -1.3442653  -1.49901742  0.09682976]
 [-1.07222061  0.20107586  1.07929083  1.43367419]
 [ 0.88257624  0.70425436  0.12504735 -0.9645607 ]]
Gen b2 [[0.]
 [0.]
 [0.]
 [0.]]
Gen W3 [[-0.00857711  0.27236544  0.01280088  1.72911103 -1.06843791 -0.13964534
   0.19486949 -0.16106157 -1.71281416  0.33216272]]
Gen b3 [[0.]]
nextGen W1 [[ 0.02715842  0.41516042 -0.56379084 -0.3289123   0.00562478  0.08614645
  -0.54755219 -1.755171  ]
 [ 1.47554773  0.23382935  1.36606843 -1.09835465 -0.

[[1.]
 [1.]
 [1.]
 [1.]]


In [42]:



'''print("Gen W1" ,bugs["bug0"]["W1"])
print("Gen b1",bugs["bug0"]["b1"])
print("Gen W2", bugs["bug0"]["W2"])
print("Gen b2", bugs["bug0"]["b2"])
print("Gen W3", bugs["bug0"]["decider_W3"])
print("Gen b3", bugs["bug0"]["decider_b3"])'''


'''print("Gen W1" ,bugs["bug22"]["W1"])
print("Gen b1",bugs["bug22"]["b1"])
print("Gen W2", bugs["bug22"]["W2"])
print("Gen b2", bugs["bug22"]["b2"])
print("Gen W3", bugs["bug22"]["decider_W3"])
print("Gen b3", bugs["bug22"]["decider_b3"])
'''



movement_points = all_the_bugs_deciding(bugs,food)
#print(movement_points)


#print("beginning: " ,bugs["bug0"]["coordinates"])
move_all_the_bugs(bugs,movement_points)
#print("last: " ,bugs["bug0"]["coordinates"])
#print("movement:" ,movement_points["bug0"])

evaluating_bugs(bugs,food)

survivor_names = natural_selection(bugs,food)
print(survivor_names)

gene_bag = the_big_shuffler(bugs,survivor_names)
#print(gene_bag)

bugs = single_cell_create(bugs,gene_bag)
'''print("nextGen W1" ,bugs["bug0"]["W1"])
print("nextGen b1",bugs["bug0"]["b1"])
print("nextGen W2", bugs["bug0"]["W2"])
print("nextGen b2", bugs["bug0"]["b2"])
print("nextGen W3", bugs["bug0"]["decider_W3"])
print("nextGen b3", bugs["bug0"]["decider_b3"])'''


bugs = two_cell_create(bugs,gene_bag)

'''print("nextGen W1" ,bugs["bug22"]["W1"])
print("nextGen b1",bugs["bug22"]["b1"])
print("nextGen W2", bugs["bug22"]["W2"])
print("nextGen b2", bugs["bug22"]["b2"])
print("nextGen W3", bugs["bug22"]["decider_W3"])
print("nextGen b3", bugs["bug22"]["decider_b3"])
'''

bugs = four_cell_create(bugs,gene_bag)




['bug4', 'bug13', 'bug15', 'bug2']


In [15]:
a = random.randint(0,7)
print(a)

gene = gene_bag["layer_1_cells_W1"][a]
#print(gene)
a = random.randint(0,7)
gene2 = gene_bag["layer_1_cells_W1"][a]
a = random.randint(0,7)
gene3 = gene_bag["layer_1_cells_W1"][a]

liste = [gene,gene2,gene3]
#final = np.concatenate((gene.reshape(1,8),gene2.reshape(1,8)),axis = 0)
#print(final.shape)
vistack = np.vstack(liste)
print(vistack.shape)

1
(3, 8)


In [12]:
a = np.array([1,1,1,1,1,1,1,1])
b = bugs["bug0"]["W1"]
c = bugs["bug0"]["W2"]
d = bugs["bug0"]["decider_W3"]
print(np.dot(d,np.dot(c,np.dot(b,a))).shape)

ValueError: shapes (1,2) and (4,) not aligned: 2 (dim 1) != 4 (dim 0)

(0,)


In [33]:
first =np.array([[1,2,3,4],[4,5,6,7],[7,8,9,10]])
print(first.shape)

a = np.array([2,2,2,2])
a = a.reshape(a.shape[0],1)
k = np.ones([first.shape[0],first.shape[1]])

second = k.T*a
print(second)
print(second.shape)

result = np.append(first.T,second,axis =0)

print(result)
print(result.shape)

(3, 4)
[[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]
(4, 3)
[[ 1.  4.  7.]
 [ 2.  5.  8.]
 [ 3.  6.  9.]
 [ 4.  7. 10.]
 [ 2.  2.  2.]
 [ 2.  2.  2.]
 [ 2.  2.  2.]
 [ 2.  2.  2.]]
(8, 3)


In [41]:
print(result[0:2,:])
print(result)

[[1. 4. 7.]
 [2. 5. 8.]]
[[ 1.  4.  7.]
 [ 2.  5.  8.]
 [ 3.  6.  9.]
 [ 4.  7. 10.]
 [ 2.  2.  2.]
 [ 2.  2.  2.]
 [ 2.  2.  2.]
 [ 2.  2.  2.]]
